# 1. Naver API를 이용한 블로그 포스트들의 URL 받아오기


- 현재로서는 가져올 수 있는 블로그 포스트 링크의 개수가 최대 1100개로 제한되어 있는 것으로 보임.<br/>(이게 걸림돌로 작용될 수 있을듯..)
- API를 이용해서 데이터를 호출할 수 있는 횟수가 하루에 2만 5천번으로 제한되어 있음.
- 현재 이태우의 API 키와 비밀번호를 이용하여 코드가 작성되어 있음 (API_PARAMETER)
- 현재는 많은 링크를 가져올 필요가 없으므로, ### 테스트를 위해 넣은 부분 ### 코드블록을 이용하여 API 호출 횟수를 제한함
<br/><br/>

In [58]:
# <Code Reference> 
# - (Github, xotrs/naver-blog-crawler) https://github.com/xotrs/naver-blog-crawler/blob/master/NaverBlogCrawler.py
# - (검색 API) https://developers.naver.com/docs/search/blog/

import os
import sys
import urllib.request
import json
import math

API_PARAMETER = {
    "client_id": "E5cL2__bFGNST8f4eknt",
    "client_secret": "kcwtUqHT98"
}

QUERY_PARAMETER = {
    "query": "카카오미니",
    "display": 100,
    "sort_type": "sim"
}

def naver_api_blog_search(api_parameter, search_word, 
                          display_count=10, start=1, sort_type='sim'):
    result = None
    
    client_id = api_parameter['client_id']
    client_secret = api_parameter['client_secret']
    
    encText = urllib.parse.quote("{}".format(search_word))
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&display=" + str(
            display_count) + "&start=" + str(start) + "&sort=" + sort_type  # json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과 예시
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode) 
    return result

def get_search_result_page_count(api_parameter, query_word, display_count):
    search_result_page_count = 0
    
    decoded_response_body = naver_api_blog_search(api_parameter, query_word)
    if decoded_response_body:
        json_response_body = json.loads(decoded_response_body)
        
        total_posts = json_response_body['total']
        if total_posts != 0:
            search_result_page_count = math.ceil(total_posts / display_count)
    return search_result_page_count
    
def get_urls(query_word, display_count, sort_type, search_result_page_count):
    url_list = []
    for i in range(search_result_page_count):
        start = display_count*i + 1
        if start > 1000:
            start = 1000
            
        decoded_response_body = naver_api_blog_search(api_parameter, query_word, display_count, start, sort_type)
        if decoded_response_body:
            json_response_body = json.loads(decoded_response_body)
            
            for dp in json_response_body['items']:
                url_list.append(dp['link'].replace("amp;", ""))
        
        ### 테스트를 위해 넣은 부분 ###
        #if i == 1:
        #    break
        #########################
        
        if start == 1000:
            break
    return url_list


if __name__ == '__main__':
    api_parameter = API_PARAMETER 
    query = QUERY_PARAMETER['query']
    display = QUERY_PARAMETER['display']
    sort_type = QUERY_PARAMETER['sort_type']
    
    p_count = get_search_result_page_count(api_parameter, query, display)
    urls = get_urls(query, display, sort_type, p_count)

In [59]:
urls

['https://blog.naver.com/ilovemommy12?Redirect=Log&logNo=221430643354',
 'https://blog.naver.com/vjrm5933?Redirect=Log&logNo=221471650353',
 'https://blog.naver.com/sejinhee?Redirect=Log&logNo=221444588938',
 'https://blog.naver.com/seva2000?Redirect=Log&logNo=221438235022',
 'https://blog.naver.com/3jwhjhy?Redirect=Log&logNo=221466001171',
 'https://blog.naver.com/nam04030?Redirect=Log&logNo=221459408391',
 'https://blog.naver.com/syokolla?Redirect=Log&logNo=221474510797',
 'https://blog.naver.com/hambak88?Redirect=Log&logNo=221453186799',
 'https://blog.naver.com/mfb5643?Redirect=Log&logNo=221516804505',
 'https://blog.naver.com/penic798?Redirect=Log&logNo=221406733164',
 'https://blog.naver.com/tlswlgns8888?Redirect=Log&logNo=221415718120',
 'https://blog.naver.com/hancm?Redirect=Log&logNo=221452577073',
 'https://blog.naver.com/dgeeri?Redirect=Log&logNo=221500433742',
 'https://blog.naver.com/yarosiku?Redirect=Log&logNo=221315149932',
 'https://blog.naver.com/byki2013?Redirect=Log&

In [60]:
len(urls)

1100

<br/><br/>
# 2. 받아온 URL들을 이용하여 원하는 정보를 scraping

In [43]:
# <Code Reference> 
# - (zilu11의 블로그 포스트) https://blog.naver.com/zilu11/221255183159

import requests
from bs4 import BeautifulSoup

import time
import datetime
import re


def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url 

def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            # e.g. 2019. 04. 01. 15:49 4시간 전
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', attrs={'id': re.compile('^post-view')}):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', 'view'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts
        
if __name__ == '__main__':
    
    
    for url in urls:    

        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)

        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)

        print('url: ', url)
        print('real_url: ', real_url)
        print()
        print('post title: ', title)
        print()
        print('publish date time: ', publish_datetime)
        print()
        print("texts: ", texts)
        print("\n-----------------------------------\n")

        time.sleep(0.01)

url:  https://blog.naver.com/gkqudgh6881?Redirect=Log&logNo=221514263292
real_url:  http://blog.naver.com/PostView.nhn?blogId=gkqudgh6881&logNo=221514263292&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  KT 기가지니 로봇청소기 아이클레보 O5 완벽정복! 누워서 명령만한다!

publish date time:  2019-04-15 17:29:00

texts:  KT 기가지니 로봇청소기 아이클레보 O5 완벽정복! 누워서 명령만한다!
KT 기가지니 로봇청소기 유진로봇 아이클레보 O5 인트로
*로봇연기 주의*
KT기가지니 로봇청소기 
아이클레보 O5
안녕하세요 병호몬입니다~~!! 
오늘은 1가구 1로봇청소기 시대에 추천드리고픈 
KT기가지니 AI 플렛폼 로봇청소기 iCLEBO O5를 
리뷰해보려합니다! 요즘 로봇청소기가 많은데 
요녀석은 기가지니와 연동이 가능하기에 KT를 이용하시고 기가지니가 있으신분이라면 추천드릴만한 녀석입니다
아이클레보 O5 개봉기
기가지니 로봇청소기 O5를 개봉하니 제일먼저 청소브러쉬와 설명서가 보이네요
청소브러쉬는 로봇청소기의 손역활늘하는 중요한 녀석이죠 그리고 설명서는 여러개가있는데 KT기가지니와 연동하는방법등 아주 상세하게 다 나와있습니다 처음사용하시는분은 꼭 꼼꼼히 읽어보시고 사용하시길 바랍니다!!
그다음은 로봇청소기 본체와 악세사리박스가 있었는데요 먼저 악세사리박스 구성품부터 살펴보겠습니다
아이클레보 O5 악세사리 박스
악세사리 박스에는 로봇청소기 리모컨과 건전지가 들어있었습니다. 건전지가 포함되어있어 따로 구입하러 갈필요가 없었는데요 이런 배려 아주 좋아요~~ 건전지는 AAA건전기 2개가 필요합니다
두번째는 충전어댑터! 220v 커넥트와 분리되어들어있었는데요 쉽게 결합할수있었습니다. 국내에서 사용하기 좋은 가정요 22

url:  https://blog.naver.com/lucky_box7?Redirect=Log&logNo=221501466522
real_url:  http://blog.naver.com/PostView.nhn?blogId=lucky_box7&logNo=221501466522&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  KT 기가지니 프로야구 정보 확인과 홈IoT로 플러그 관리까지!

publish date time:  2019-03-30 22:05:00

texts:  KT 기가지니 프로야구 정보 확인과 홈IoT로 플러그 관리까지!
업그레이드된 기가지니 ?!
기가지니는 KT의 인공지능 스피커이자, 올레TV 재생이 가능한 셋톱박스입니다. 과거 출시와 동시에 큰 인기를 얻었고 현재는 엄청난 가입자를 보유하고 있는데요. 정말 다양한 VOD 콘텐츠가 있고 아이들을 위한 교육용 콘텐츠도 있어 현재도 유저들이 늘어나고 있는 상황입니다. 오늘은 이 기가지니의 업데이트 소식을 전해드릴게요.
먼저 프로야구 개막에 맞춰 한국 프로야구 음성안내 서비스가 추가됐습니다. 사실 작년 10월, 11월 포스트시즌에 제한적으로 서비스를 운영하기도 했는데요. 3월26일부터 정식 서비스 업데이트가 진행되어 프로야구에 대한 질문을 음성으로 물어보고, 소식을 알 수 있게 됐습니다.
경기 일정과 경기 결과, 현재 상황까지 음성으로 체크 !
보통 AI 스피커에 날씨가 어떤지 물어보고, 재생중인 음악을 끄는 등으로 많이 활용하죠? 프로야구 음성안내 서비스도 비슷합니다. 예를들어 일정이 궁금할 땐 "내일 프로야구 언제해?", "내일 KT위즈 경기일정 알려줘" 등으로 확인할 수 있고 현재 경기가 어떻게 진행되고 있는지 궁금하다면 "지금 야구 어떻게 되고 있어?"로 확인 가능합니다. 그 외에 팀 순위 체크나 특정 선수에 대한 정보까지 물어볼 수도 있죠. 
꼭 화면을 보고있지 않더라도 청소를 하거나, 다른 작업을 

url:  https://blog.naver.com/newswire1?Redirect=Log&logNo=221525101621
real_url:  http://blog.naver.com/PostView.nhn?blogId=newswire1&logNo=221525101621&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  KT, 집안 어디서나 즐기는 ‘기가지니 테이블TV’ 첫 선

publish date time:  2019-04-29 17:30:12 6시간 전

texts:  성남--(뉴스와이어) 2019년 04월 29일 -- KT가 화면과 셋톱박스를 결합해 편의성을 높인 ‘일체형 인공지능 TV’를 선보이며 TV 생활의 중심을 가족에서 개인으로 이동시킨다고 29일 밝혔다. 또한 인공지능(AI) 개인화 서비스를 본격화하며 AI가 보다 생활 깊숙이 자리 잡도록 했다.  ◇국내 최초 일체형 인공지능 TV 공개, AI 개인화 서비스 등 사업전략 발표  KT(회장 황창규)가 서울 종로구 KT광화문빌딩 West에서 기자설명회를 열고 국내 최초 일체형 AI TV ‘기가지니 테이블TV’와 AI 사업전략을 29일 발표했다. 이번 행사에는 AI사업단장 김채희 상무, AI기술담당 최준기 상무 등 KT에서 AI 사업을 주도하는 임직원들이 참석했다. 이날 KT는 AI 서비스 ‘기가지니’ 가입자가 165만을 넘어섰다며 올해 3분기 안으로 가입자 200만 달성을 예상했다.  KT가 2017년 국내 최초로 선보인 AI TV(셋톱박스) ‘기가지니’는 청각뿐 아니라 시청각에 기반한 사용자환경(UI)으로 호평을 받았다. 직관적인 이용이 가능해 AI기기는 젊은 세대의 전유물이라는 고정관념에서 벗어나 중장년층이나 어린이도 쉽게 AI를 즐기는 문화를 만들었다는 평가를 받았다.  이번에 선보인 ‘기가지니 테이블TV’는 셋톱박스에 화면(디스플레이)을 결합해 개인용 AI TV로 다양한 활용이 예상된다. 스마트패드와

url:  https://blog.naver.com/cshee32?Redirect=Log&logNo=221470736312
real_url:  http://blog.naver.com/PostView.nhn?blogId=cshee32&logNo=221470736312&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  기가지니 홈트레이닝 시작해줘! KT AI 스피커 함께 PT 중~

publish date time:  2019-02-21 07:00:00

texts:  기가지니 홈트레이닝 시작해줘! KT AI 스피커 함께 PT 중~ 
거의 집에서만 생활하다 보니 운동량이 정말 부족해요!
억지로라도 움직여야겠다는 생각이 들어 작년 여름에 아파트 내 헬스장도 등록했는데 일이 바빠지니 잘 안 가게 되고, 트레이너가 없어서 부위별 집중 운동을 할 수 없다는 게 참 아쉬웠어요.
따로 1:1 혹은 그룹 PT를 받고 싶지만 비용도 비싸고, 자주 못 갈까 봐 망설였고요.
그래서 홈트레이닝 하자는 마음으로 짬짬이 계단도 오르고, 스쿼트나 스트레칭도 했지만 운동을 제대로 하는 게 맞나? 의문이 생기더라고요. 요즘 홈트하는 사람들이 늘어나며 관련된 앱도 많이 있지만, 작은 스마트폰 화면을 보며 따라 하는 것도 한계! 중간중간 정지시켰다가 다시 재생하며 따라 하려니 맥이 끊기더라고요.
사실 작년에 반짝 운동했을 때 건강해지는 기분도 들었지만 살도 빠지니 기분이 참 좋았었는데~ 운동 끊고 예전처럼 생활하니 몸무게가 다시 돌아오더라고요. 안 되겠다 싶어서 최근 운동복과 홈트레이닝 위한 기구도 몇 가지 구매해서 다시 시작해보기로 했습니다.
KT 기가지니 AI 스피커와 함께요^^
AI스피커 기가지니와 모바일 헬스케어 업체인 눔(NOOM)이 홈트레이닝 서비스를 시작했습니다!
다양한 운동 영상과 함께 테마별 프로그램을 적용해서 목표를 달성할 수 있기 때문에 PT 받는 것처럼 체계적인 운동을 할 수 있는데요.

url:  https://blog.naver.com/1501fox?Redirect=Log&logNo=221459621466
real_url:  http://blog.naver.com/PostView.nhn?blogId=1501fox&logNo=221459621466&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  kt 기가지니 인공지능 스피커 사용해보니 정말 놀랍고 편해요 ~

publish date time:  2019-02-06 23:59:00

texts:  kt 기가지니 인공지능 스피커 사용해보니 정말 놀랍고 편해요 ~
일주일전  신혼때부터 오랫동안 함께했던  sk 의 ip 티비와 작별을 했습니다 ~~   여러가지 이유가 있었지만 최근에 너무많은 오류로 골치가 아파서 결국 오랫동안 함께한녀석을 정리했어요 ~~
바꿀땐 과감하게 미련없이  ~~~   
여기저기  많은 지인분들에게 물어보고  상의한끝에  kt 로  변경을 하기로 했는데요  ,
 정말 돈안들이고  최신 인공지능스피커까지  사용이 가능하다고 해서  너무 놀랐어요 ~~
  그동안 난  무얼하고있었던건지  , 왜 나만  바보같이모르고 있었던건지  ~~   후회도 되더라고요   
이녀석이 이번에 우리집에온  기가지니  인공지능스피커 입니다  !!!    하루종이 집에서  어른애들 할거 없이  
지니야 ~~  를 왜치게 만드는데요  , 편리한 기능이 너무많고  , 특히  목소리 인식이 잘되서 너무나 놀랐습니다  !!!
기계에 대한 사용법을 잘익혀야 조금이라도 더  활용도가 높을것 같아서  받아둔 간편사용설명서와  기가지니사용설명서를  꺼내보았습니다 ~~
일단 가장먼저 간편사용설명서를  보고  이것저것 따라 해봤는데요  ,  쉽게 설명이 돼어있어서  금방 따라할수 있어서 너무 편했어요  , 사실 이런걸 보면 하기도전에 미리  겁먹는 편이라서  ^^:   그런데 이번엔 너무  쉽게  따라할수있었어요  
일반적으로 사

url:  https://blog.naver.com/snfbby08?Redirect=Log&logNo=221319612466
real_url:  http://blog.naver.com/PostView.nhn?blogId=snfbby08&logNo=221319612466&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  인공지능스피커 KT 기가지니2 설치 및 사용기

publish date time:  2018-07-15 21:01:00

texts:  인공지능스피커 KT 기가지니2 설치 및 사용기
최근 '소리동화 읽어줘~'하고 책읽기를 시작하면동화에 나오는 배경음을 들려주는 KT기가지니 광고를 보고 '우와~!' 하고 감탄했던 생각이 나네요 ^^예원이도 광고를 잘 보는 편이라 따라서 '소리동화 읽어줘!'하고 이야기하길래저희집에도 인공지능스피커 KT 기가지니2를 설치했습니다 !!
인공지능스피커 기가지니2 택배를 받자마자 드디어 우리집도 스마트해지는구나~ 생각했어요 ㅎㅎ
명칭이 인공지능스피커이지만 스피커에 장착된 마이크를 통해 음성 명령이 가능하고,블루투스 페어링을 통해 스피커를 사용할 수도 있습니다 ~
박스를 오픈하니 가장 먼저 사용설명서가 눈에 띄었어요!
구성품은 다음과 같습니다 !HDMI 케이블을 이용해 TV와 기가지니를 연결하는 방식입니다.
기기의 앞면입니다.앞면 세로 중앙부에 LED 상태표시등이 설치되어 있어요.
살펴보니 유명한 하만카돈이더라고요 ~최근에 BMW 구입을 생각중인데 차에 하만카돈이 적용되어 있어서 호감을 가지고 있었는데집에서 먼저 하만카돈 스피커를 사용하게 되었네요 : ))
기기의 윗면에는 볼륨 조절버튼과 블루투스 연결 시 페어링 버튼이 있어요.윗부분에 음성입력차단 버튼은 기가지니 호출어 인식을 잠시 멈추고 싶을 때 음성입력을 차단하는 버튼입니다.가운데 동작/대기 모드 전환버튼은 기기를 대기모드와 동작모드로 전환해줍니다.
기기 옆면에서 USB와 전원 ON

KeyboardInterrupt: 

<br/><br/>
# 3. 받아온 URL들을 이용하여 원하는 정보를 scraping 하고 CSV로 저장

In [61]:
# <Code Reference> 
# - (zilu11의 블로그 포스트) https://blog.naver.com/zilu11/221255183159

import requests
from bs4 import BeautifulSoup

import time
import datetime
import re


def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url 

def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            # e.g. 2019. 04. 01. 15:49 4시간 전
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', attrs={'id': re.compile('^post-view')}):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', 'view'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

if __name__ == '__main__':
    title_list = []
    publish_datetime_list = []
    text_list = []
    
    for i, url in enumerate(urls):    

        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)
        title_list.append(title)
        
        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        publish_datetime_list.append(publish_datetime)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)
        text_list.append(texts)
        
        print("{}.".format(i), "finished")
        time.sleep(0.01)

0. finished
1. finished
2. finished
3. finished
4. finished
5. finished
6. finished
7. finished
8. finished
9. finished
10. finished
11. finished
12. finished
13. finished
14. finished
15. finished
16. finished
17. finished
18. finished
19. finished
20. finished
21. finished
22. finished
23. finished
24. finished
25. finished
26. finished
27. finished
28. finished
29. finished
30. finished
31. finished
32. finished
33. finished
34. finished
35. finished
36. finished
37. finished
38. finished
39. finished
40. finished
41. finished
42. finished
43. finished
44. finished
45. finished
46. finished
47. finished
48. finished
49. finished
50. finished
51. finished
52. finished
53. finished
54. finished
55. finished
56. finished
57. finished
58. finished
59. finished
60. finished
61. finished
62. finished
63. finished
64. finished
65. finished
66. finished
67. finished
68. finished
69. finished
70. finished
71. finished
72. finished
73. finished
74. finished
75. finished
76. finished
77. finis

594. finished
595. finished
596. finished
597. finished
598. finished
599. finished
600. finished
601. finished
602. finished
603. finished
604. finished
605. finished
606. finished
607. finished
608. finished
609. finished
610. finished
611. finished
612. finished
613. finished
614. finished
615. finished
616. finished
617. finished
618. finished
619. finished
620. finished
621. finished
622. finished
623. finished
624. finished
625. finished
626. finished
627. finished
628. finished
629. finished
630. finished
631. finished
632. finished
633. finished
634. finished
635. finished
636. finished
637. finished
638. finished
639. finished
640. finished
641. finished
642. finished
643. finished
644. finished
645. finished
646. finished
647. finished
648. finished
649. finished
650. finished
651. finished
652. finished
653. finished
654. finished
655. finished
656. finished
657. finished
658. finished
659. finished
660. finished
661. finished
662. finished
663. finished
664. finished
665. f

In [62]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 'publish_datetime': publish_datetime_list, 'text': text_list, 'url': urls})

In [63]:
df = df[['title', 'publish_datetime', 'text', 'url']]

In [64]:
df

,title,publish_datetime,text,url
0,헤이 카카오 미니c 사용후기,2018-12-30 22:37:00,헤이 카카오 미니c 사용후기\n헤이 카카오 미니c 사용후기\n친구한테 \n카카오미니...,https://blog.naver.com/ilovemommy12?Redirect=L...
1,딜라이브 플러스 UHD 카카오 미니 시간 가는 줄 몰라요,2019-02-21 21:26:00,딜라이브 플러스 UHD 카카오 미니 시간 가는 줄 몰라요\n요즘 매일 잠들기 전에 ...,https://blog.naver.com/vjrm5933?Redirect=Log&l...
2,딜라이브플러스 UHD와 카카오미니의 조화 아주 조쿠먼,2019-01-17 20:07:00,딜라이브플러스 UHD와 카카오미니의 조화 아주 조쿠먼\n딜라이브플러스 UHD & ...,https://blog.naver.com/sejinhee?Redirect=Log&l...
3,딜라이브플러스UHD & 카카오미니 이 조합 인정!,2019-01-09 22:00:00,딜라이브플러스UHD & 카카오미니 이 조합 인정! \n저희 부부는 집에서 영화나 드...,https://blog.naver.com/seva2000?Redirect=Log&l...
4,딜라이브플러스UHD 카카오미니 함께 하고 볼거리 풍족하게,2019-02-14 21:07:00,딜라이브플러스UHD 카카오미니 함께 하고 볼거리 풍족하게 \n예전 같으면 하원하고 ...,https://blog.naver.com/3jwhjhy?Redirect=Log&lo...
5,딜라이브플러스UHD & 카카오미니 덕분에 문화생활 즐겨요~,2019-02-11 23:30:00,딜라이브플러스UHD & 카카오미니 덕분에 문화생활 즐겨요~\n늘 바쁘다 바빠를 입에...,https://blog.naver.com/nam04030?Redirect=Log&l...
6,딜라이브플러스UHD 카카오미니 언제 어디서든 드라마 영화를 보자!,2019-02-25 17:57:00,딜라이브플러스UHD 카카오미니 언제 어디서든 드라마 영화를 보자!\n안녕하세요 잇님...,https://blog.naver.com/syokolla?Redirect=Log&l...
7,자취 필수템 딜라이브 플러스 UHD & 카카오미니 리얼 사용후기,2019-01-28 19:22:00,자취 필수템 \n딜라이브 플러스 UHD & 카카오미니 리얼 사용후기 \n자취 필수템...,https://blog.naver.com/hambak88?Redirect=Log&l...
8,카카오 프렌즈 미니가습기 라이언 귀요미♡,2019-04-18 15:34:00,카카오 프렌즈 미니가습기 라이언 귀요미♡\n카카오 프렌즈 미니가습기 라이언 귀요미\...,https://blog.naver.com/mfb5643?Redirect=Log&lo...
9,"""무엇이든 물어봐!"" 카카오미니C",2018-11-26 21:21:00,우리집은 지금 #헤이카카오열풍이다.6살 9살 두아이들이집에만 오면 헤이카카오를 외...,https://blog.naver.com/penic798?Redirect=Log&l...


In [65]:
df.to_csv('../data/naver_blog_kakaomini.csv')

<br/><br/><br/>
# 예외> 본문 데이터가 중복돼서 가져와지는 것 확인하기

In [74]:
url = "https://blog.naver.com/mchan21/221249916216"
#url = "http://blog.naver.com/PostView.nhn?blogId=yukihana96&logNo=221389690746&from=search&redirect=Log&widgetTypeCall=true&directAccess=false"
real_url = get_realdata_url_naverblog(url)
r = requests.get(real_url)
soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

In [75]:
real_url

'http://blog.naver.com/PostView.nhn?blogId=mchan21&logNo=221249916216&redirect=Dlog&widgetTypeCall=true&directAccess=false'

In [76]:
# 블로그 텍스트 추출
texts = get_texts_naverblog(soup)
print(texts)


 이건 사야해! 네이버 클로바 미니언즈 스피커 개봉기슈퍼배드의 귀여운 악당! 색다른 언어와 아이같은 행동이 특징인 귀여운 미니언즈를 모르는 분들이 없을겁니다.노란색에 바지를 입고 하나 또는 2개의 눈을 가졌으며 고글을 하고 있는 미니언즈가 네이버 클로바 프렌즈와 헙업을 통해 인공지능 블루투스 스피커로 출시되었습니다.저는 평소 네이버 뮤직을 사용해서 네이버에서 선보이는 인공지능 스피커 클로바 제품에 굉장히 관심이 많았는데요.이번에 새롭게 출시된 클로바 프렌즈 미니언즈 에디션의 경우 정기결제 이용자에게 권장소비자가 134000원 제품을 59% 할인으로 55000원에 구입할 수 있어서 바로 주문했습니다.참고로 신규 가입자에게는 48% 할인가 94900원(무제한듣기 6개월 약정 제공), 선물시 45% 할인가 87000원(무제한듣기 3개월 쿠폰 포함)으로 구입이 가능합니다.​ 기존 클로바 프렌즈의 경우 라인의 인기 캐릭터인 브라운과 샐리를 형상화한 디자인으로 출시 당시 이틀만에 1만대가 모두 소진될 만큼 큰 인기를 끌었습니다.​ 저도 네이버의 AI 스피커 프렌즈를 사용하고 있는데요. 평소 좋아하는 캐릭터 브라운을 형상화한 제품으로 저는 매일 아침 라인 프렌즈를 통해 일어나고, 출근 전 날씨정보, 음악감상을 하다가 퇴근 후에는 일정 관리나 브리핑, 스포츠 경기결과 등을 물어보고 자장가를 들으며 잠을 자는 등 하루를 함께하고 있습니다.패키지 박스는 제품 디자인을 제외하면 거의 비슷한 느낌을 받을 수 있습니다.​ 패키지 박스를 열면 귀여운 네이버 클로바 미니언즈 본체를 확인할 수 있습니다. 그 밑으로 구성품들이 빼곡히 들어있습니다.​ 구성품은 심플한데요. 클로바 프렌즈 미니언즈 본체와 충전 어답터, 충전 케이블, 퀵 스타트 가이드 및 품질보증서, 발화문 가이드, 표정을 꾸밀 수 있는 미니언즈 페이스 스티커 등으로 구성되어 있습니다.​ 사용설명서에는 제품의 제원부터 특징, 연결방법, 사용방법, 사용팁 등 상세하게 설명되어 사용전 미리 읽어보면 누구나 쉽게 사용할 수 있을 것